# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/thesis-folder')

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sc
import datetime
import calendar

In [ ]:
pip install --upgrade networkx

     |████████████████████████████████| 1.9 MB 13.9 MB/s 
  Attempting uninstall: networkx
    Found existing installation: networkx 2.5.1
    Uninstalling networkx-2.5.1:
      Successfully uninstalled networkx-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install python-louvain

In [ ]:
import networkx as nx
from community import community_louvain
import json

# Louvain implementation

In [ ]:
#Read the season wise dataframes created before in pre-processing
domestic_spring_df = pd.read_csv('intermittent-result/Domestic US/dom_spring.csv')
domestic_summer_df = pd.read_csv('intermittent-result/Domestic US/dom_summer.csv')
domestic_autumn_df = pd.read_csv('intermittent-result/Domestic US/dom_autumn.csv')
domestic_winter_df = pd.read_csv('intermittent-result/Domestic US/dom_winter.csv')

## Summer

In [ ]:
domestic_summer_df.head()

,Unnamed: 0,year,month,origin_city,dest_city,month_num,passengers count
0,11342,2005,august,aberdeen,brainerd,8,1.0
1,11343,2005,august,aberdeen,manhattan/ft. riley,8,78.0
2,11344,2005,august,aberdeen,minneapolis,8,2291.0
3,11345,2005,august,aberdeen,peoria,8,1.0
4,11346,2005,august,aberdeen,pierre,8,9.0


Creating graph

In [ ]:
 domestic_graph = nx.from_pandas_edgelist(domestic_summer_df,'origin_city','dest_city',
                                  edge_attr= 'passengers count',
                                  create_using=nx.MultiGraph())

In [ ]:
len(domestic_graph.nodes)

1467

Creating partitions

creating and storing the partition as json

In [ ]:
'''summer_partition  = community_louvain.best_partition(domestic_graph)
community_louvain.modularity(summer_partition, domestic_graph)

#code to save partition as json file
partition_dict = json.dumps(summer_partition)
f = open("intermittent-result/Domestic US/dom_sm_part_9.json", "w")
f.write(partition_dict)
f.close()'''

Retreiving the stored partition

In [ ]:
#code to retrieve partition from json
f = open(os.path.join("intermittent-result/Domestic US/dom_sm_part_9.json"),"r")
dom_sm_partition = json.load(f)
print("Partition length: {}".format(len(dom_sm_partition)))

Partition length: 1467


In [ ]:
summer_mod_value = community_louvain.modularity(dom_sm_partition, domestic_graph)

In [ ]:
print('Modularity score for summer partition is:{}'.format(summer_mod_value))

Modularity score for summer partition is:0.34522778648656915


Retrieving community details and merging them with the summer dataframe

In [ ]:
#partition dictionary to dataframe
dom_summer_partition_df = pd.DataFrame.from_dict(dom_sm_partition, orient='index')
dom_summer_partition_df.reset_index(inplace=True)
dom_summer_partition_df.rename(columns={"index":"city",0:"community"}, inplace=True)
print(dom_summer_partition_df['community'].unique())
dom_summer_partition_df.head()

[0 1 2 5 6 4 3]


,city,community
0,aberdeen,0
1,brainerd,0
2,manhattan/ft. riley,0
3,minneapolis,0
4,peoria,1


In [ ]:
#merge for origin city to community
dom_summer_community_df = pd.merge(dom_summer_partition_df, domestic_summer_df, how='left', left_on='city', right_on='origin_city')


In [ ]:
dom_summer_community_df.shape
dom_summer_community_df.dropna(inplace=True)
#dom_summer_community_df.drop(columns=['origin_city','dest_city'], inplace=True)

dom_summer_community_df = dom_summer_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_summer_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2010.0,august,1.0
1,47-mile mine,2,2010.0,july,8.0
2,47-mile mine,2,2010.0,june,6.0
3,aberdeen,0,2005.0,august,2380.0
4,aberdeen,0,2005.0,july,2449.0


In [ ]:
dom_summer_community_df.isna().sum()

city                0
community           0
year                0
month               0
passengers count    0
dtype: int64

In [ ]:
#merge for destination city to community
dom_summer_community_df1 = pd.merge(dom_summer_partition_df, domestic_summer_df, how='left', left_on='city', right_on='dest_city')

In [ ]:
dom_summer_community_df1.drop(columns=['origin_city','dest_city'], inplace=True)
dom_summer_community_df1

dom_summer_community_df1 = dom_summer_community_df1.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
#dom_summer_community_df.rename(columns = {'us_lat':'latitude', 'us_long':'longitude'}, inplace=True)
dom_summer_community_df1.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005.0,august,6.0
1,47-mile mine,2,2010.0,august,6.0
2,47-mile mine,2,2010.0,july,1.0
3,47-mile mine,2,2010.0,june,6.0
4,47-mile mine,2,2012.0,august,5.0


In [ ]:
#concatenating both the above merges
dom_summer_community_df = pd.concat([dom_summer_community_df, dom_summer_community_df1], axis=0)
dom_summer_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2010.0,august,1.0
1,47-mile mine,2,2010.0,july,8.0
2,47-mile mine,2,2010.0,june,6.0
3,aberdeen,0,2005.0,august,2380.0
4,aberdeen,0,2005.0,july,2449.0


In [ ]:
dom_summer_community_df.city.nunique()

1467

In [ ]:
dom_summer_community_df = dom_summer_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_summer_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005.0,august,6.0
1,47-mile mine,2,2010.0,august,7.0
2,47-mile mine,2,2010.0,july,9.0
3,47-mile mine,2,2010.0,june,12.0
4,47-mile mine,2,2012.0,august,5.0


In [ ]:
dom_summer_community_df = dom_summer_community_df.astype({"year":'int'})

In [ ]:
dom_summer_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005,august,6.0
1,47-mile mine,2,2010,august,7.0
2,47-mile mine,2,2010,july,9.0
3,47-mile mine,2,2010,june,12.0
4,47-mile mine,2,2012,august,5.0


In [ ]:
dom_summer_community_df.city.nunique()

1467

In [ ]:
#creating summary df to store the number of cities and passengers present in each community
dom_agg_dict = {'city':'nunique', 'passengers count':'sum'}
dom_summer_summary_df = dom_summer_community_df.groupby(['community'], as_index=False).agg(dom_agg_dict)
dom_summer_summary_df = dom_summer_summary_df.sort_values(by=['city','passengers count'], ascending=False)
dom_summer_summary_df

,community,city,passengers count
2,2,414,3.209578e+09
4,4,384,4.188456e+07
1,1,378,2.038474e+09
6,6,163,1.244173e+08
8,8,71,8.185341e+06
3,3,51,1.947961e+08
7,7,2,4.000000e+00
0,0,2,0.000000e+00
5,5,2,0.000000e+00


In [ ]:
dom_summer_summary_df[(dom_summer_summary_df['city']>5) & (dom_summer_summary_df['passengers count']>100)]


,community,city,passengers count
2,2,414,3.209578e+09
4,4,384,4.188456e+07
1,1,378,2.038474e+09
6,6,163,1.244173e+08
8,8,71,8.185341e+06
3,3,51,1.947961e+08


String community and summary details as csv files for future use

In [ ]:
dom_summer_community_df.to_csv('intermittent-result/Domestic US/Domestic summer community.csv', index=False)

In [ ]:
dom_summer_summary_df.to_csv('intermittent-result/Domestic US/Domestic summer summary.csv', index=False)

## Spring

In [ ]:
domestic_spring_df.head()

,Unnamed: 0,year,month,origin_city,dest_city,month_num,passengers count
0,0,2005,april,aberdeen,jamestown,4,0.0
1,1,2005,april,aberdeen,minneapolis,4,2030.0
2,2,2005,april,aberdeen,pierre,4,15.0
3,3,2005,april,aberdeen,sioux falls,4,0.0
4,4,2005,april,aberdeen,watertown,4,2.0


Graph creation

In [ ]:
spring_graph = nx.from_pandas_edgelist(domestic_spring_df,'origin_city','dest_city',
                                  edge_attr= 'passengers count',
                                  create_using=nx.MultiGraph())

Findind and storing best partition as json

In [ ]:
'''spring_partition  = community_louvain.best_partition(spring_graph)
community_louvain.modularity(spring_partition, spring_graph)

#code to save partition as json file
import json
partition_dict = json.dumps(spring_partition)
f = open("intermittent-result/Domestic US/dom_sp_part_7.json", "w")
f.write(partition_dict)
f.close()'''

Retrieving the stored partition

In [ ]:
#code to retrieve partition from json
f = open(os.path.join("intermittent-result/Domestic US/dom_sp_part_7.json"),"r")
dom_sp_partition = json.load(f)
print("Partition length: {}".format(len(dom_sp_partition)))

Partition length: 1376


In [ ]:
spring_mod_value = community_louvain.modularity(dom_sp_partition, spring_graph)
print('Modularity score for spring partition is:{}'.format(spring_mod_value))

Modularity score for spring partition is:0.33989706930074215


In [ ]:
#partition dictionary to dataframe
dom_spring_partition_df = pd.DataFrame.from_dict(dom_sp_partition, orient='index')
dom_spring_partition_df.reset_index(inplace=True)
dom_spring_partition_df.rename(columns={"index":"city",0:"community"}, inplace=True)
print(dom_spring_partition_df['community'].unique())
dom_spring_partition_df.head()

[0 2 1 5 3 4]


,city,community
0,aberdeen,0
1,jamestown,0
2,minneapolis,0
3,pierre,0
4,sioux falls,0


Creating community df

In [ ]:
#merge for origin city to community
dom_spring_community_df = pd.merge(dom_spring_partition_df, domestic_spring_df, how='right', left_on='city', right_on='origin_city')


In [ ]:
dom_spring_community_df.drop(columns=['origin_city','dest_city'], inplace=True)

dom_spring_community_df = dom_spring_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_spring_community_df.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,april,2047.0
1,aberdeen,0,2005,march,2246.0
2,aberdeen,0,2005,may,2320.0
3,aberdeen,0,2006,april,1845.0
4,aberdeen,0,2006,march,1953.0


In [ ]:
#merge for destination city to community
dom_spring_community_df1 = pd.merge(dom_spring_partition_df, domestic_spring_df, how='right', left_on='city', right_on='dest_city')
dom_spring_community_df1.shape


(488798, 9)

In [ ]:
dom_spring_community_df1.drop(columns=['origin_city','dest_city'], inplace=True)
dom_spring_community_df1 = dom_spring_community_df1.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_spring_community_df1.head()

,city,community,year,month,passengers count
0,47-mile mine,5,2010,may,0.0
1,aberdeen,0,2005,april,2115.0
2,aberdeen,0,2005,march,2400.0
3,aberdeen,0,2005,may,2306.0
4,aberdeen,0,2006,april,1875.0


In [ ]:
#concatenating both the above merges
dom_spring_community_df = pd.concat([dom_spring_community_df, dom_spring_community_df1], axis=0)
dom_spring_community_df.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,april,2047.0
1,aberdeen,0,2005,march,2246.0
2,aberdeen,0,2005,may,2320.0
3,aberdeen,0,2006,april,1845.0
4,aberdeen,0,2006,march,1953.0


In [ ]:
dom_spring_community_df = dom_spring_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_spring_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,5,2010,may,0.0
1,aberdeen,0,2005,april,4162.0
2,aberdeen,0,2005,march,4646.0
3,aberdeen,0,2005,may,4626.0
4,aberdeen,0,2006,april,3720.0


Creating summary df

In [ ]:
dom_agg_dict = {'city':'nunique', 'passengers count':'sum'}
dom_spring_summary_df = dom_spring_community_df.groupby(['community'], as_index=False).agg(dom_agg_dict)
dom_spring_summary_df = dom_spring_summary_df.sort_values(by=['city','passengers count'], ascending=False)
dom_spring_summary_df

,community,city,passengers count
2,2,566,3.228177e+09
1,1,354,1.855921e+09
5,5,343,2.782880e+07
3,3,62,4.166198e+06
0,0,49,1.726110e+08
4,4,2,0.000000e+00


In [ ]:
dom_spring_summary_df[(dom_spring_summary_df['city']>5) & (dom_spring_summary_df['passengers count']>100)]


,community,city,passengers count
2,2,566,3.228177e+09
1,1,354,1.855921e+09
5,5,343,2.782880e+07
3,3,62,4.166198e+06
0,0,49,1.726110e+08


creating csv files for future use

In [ ]:
dom_spring_community_df.to_csv('intermittent-result/Domestic US/Domestic spring community.csv', index=False)

In [ ]:
dom_spring_summary_df.to_csv('intermittent-result/Domestic US/Domestic spring summary.csv', index=False)

## Autumn

In [ ]:
domestic_autumn_df.shape

(494530, 7)

In [ ]:
domestic_autumn_df.head()

,Unnamed: 0,year,month,origin_city,dest_city,month_num,passengers count
0,104154,2005,november,aberdeen,minneapolis,11,2484.0
1,104155,2005,november,aberdeen,sioux falls,11,0.0
2,104156,2005,november,abilene,dallas/fort worth,11,5224.0
3,104157,2005,november,abilene,houston,11,1520.0
4,104158,2005,november,abilene,lubbock,11,0.0


Graph Creation

In [ ]:
autumn_graph = nx.from_pandas_edgelist(domestic_autumn_df,'origin_city','dest_city',
                                  edge_attr= 'passengers count',
                                  create_using=nx.MultiGraph())

Finding and storing best partition as json file

In [ ]:
'''autumn_partition  = community_louvain.best_partition(autumn_graph)
community_louvain.modularity(autumn_partition, autumn_graph)
print(len(set(autumn_partition.values())))

#code to save partition as json file
partition_dict = json.dumps(autumn_partition)
f = open("intermittent-result/Domestic US/dom_autumn_part_6.json", "w")
f.write(partition_dict)
f.close()'''


Retrieving the stored partition

In [ ]:
#code to retrieve partition from json
f = open(os.path.join("intermittent-result/Domestic US/dom_autumn_part_6.json"),"r")
dom_aut_partition = json.load(f)
print("Partition length: {}".format(len(dom_aut_partition)))

Partition length: 1441


In [ ]:
autumn_mod_value = community_louvain.modularity(dom_aut_partition, autumn_graph)
print('Modularity score for autumn partition is:{}'.format(autumn_mod_value))

Modularity score for autumn partition is:0.34636480329144553


Partition to dataframe

In [ ]:
#partition dictionary to dataframe
dom_aut_partition_df = pd.DataFrame.from_dict(dom_aut_partition, orient='index')
dom_aut_partition_df.reset_index(inplace=True)
dom_aut_partition_df.rename(columns={"index":"city",0:"community"}, inplace=True)
print(dom_aut_partition_df['community'].unique())
dom_aut_partition_df.head()

[0 1 2 3 5 4]


,city,community
0,aberdeen,0
1,minneapolis,0
2,sioux falls,0
3,abilene,0
4,dallas/fort worth,1


Creation of community df

In [ ]:
#merge for origin city to community
dom_aut_community_df = pd.merge(dom_aut_partition_df, domestic_autumn_df, how='right', left_on='city', right_on='origin_city')


In [ ]:
dom_aut_community_df.isna().sum()

city                0
community           0
Unnamed: 0          0
year                0
month               0
origin_city         0
dest_city           0
month_num           0
passengers count    0
dtype: int64

In [ ]:
dom_aut_community_df.drop(columns=['origin_city','dest_city'], inplace=True)

dom_aut_community_df = dom_aut_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_aut_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005,september,13.0
1,47-mile mine,2,2006,september,19.0
2,47-mile mine,2,2010,september,23.0
3,47-mile mine,2,2011,september,5.0
4,47-mile mine,2,2012,september,7.0


In [ ]:
#merge for destination city to community
dom_aut_community_df1 = pd.merge(dom_aut_partition_df, domestic_autumn_df, how='right', left_on='city', right_on='dest_city')


In [ ]:
dom_aut_community_df1.isna().sum()

city                0
community           0
Unnamed: 0          0
year                0
month               0
origin_city         0
dest_city           0
month_num           0
passengers count    0
dtype: int64

In [ ]:
dom_aut_community_df1.drop(columns=['origin_city','dest_city'], inplace=True)

dom_aut_community_df1 = dom_aut_community_df1.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_aut_community_df1.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005,september,18.0
1,47-mile mine,2,2006,september,10.0
2,47-mile mine,2,2010,september,12.0
3,47-mile mine,2,2011,september,1.0
4,47-mile mine,2,2012,september,4.0


In [ ]:
#concatenating both the above merges
dom_aut_community_df = pd.concat([dom_aut_community_df, dom_aut_community_df1], axis=0)
dom_aut_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005,september,13.0
1,47-mile mine,2,2006,september,19.0
2,47-mile mine,2,2010,september,23.0
3,47-mile mine,2,2011,september,5.0
4,47-mile mine,2,2012,september,7.0


In [ ]:
dom_aut_community_df = dom_aut_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_aut_community_df.head()

,city,community,year,month,passengers count
0,47-mile mine,2,2005,september,31.0
1,47-mile mine,2,2006,september,29.0
2,47-mile mine,2,2010,september,35.0
3,47-mile mine,2,2011,september,6.0
4,47-mile mine,2,2012,september,11.0


creation of summary df

In [ ]:
dom_agg_dict = {'city':'nunique', 'passengers count':'sum'}
dom_aut_summary_df = dom_aut_community_df.groupby(['community'], as_index=False).agg(dom_agg_dict)
dom_aut_summary_df = dom_aut_summary_df.sort_values(by=['city','passengers count'], ascending=False)
dom_aut_summary_df

,community,city,passengers count
0,0,423,1.864460e+09
1,1,422,3.003182e+09
2,2,362,2.731003e+07
3,3,169,1.095363e+08
5,5,63,3.993491e+06
4,4,2,9.211000e+03


In [ ]:
dom_aut_summary_df[(dom_aut_summary_df['city']>5) & (dom_aut_summary_df['passengers count']>100)]


,community,city,passengers count
0,0,423,1.864460e+09
1,1,422,3.003182e+09
2,2,362,2.731003e+07
3,3,169,1.095363e+08
5,5,63,3.993491e+06


In [ ]:
dom_aut_community_df.to_csv('intermittent-result/Domestic US/Domestic autumn community.csv', index=False)

In [ ]:
dom_aut_summary_df.to_csv('intermittent-result/Domestic US/Domestic autumn summary.csv', index=False)

## Winter

In [ ]:
domestic_winter_df.shape

(492744, 7)

In [ ]:
domestic_winter_df.head()

,Unnamed: 0,year,month,origin_city,dest_city,month_num,passengers count
0,23313,2005,december,aberdeen,minneapolis,12,2344.0
1,23314,2005,december,aberdeen,sioux falls,12,0.0
2,23315,2005,december,abilene,austin,12,0.0
3,23316,2005,december,abilene,dallas/fort worth,12,5665.0
4,23317,2005,december,abilene,houston,12,1506.0


Graph creation

In [ ]:
winter_graph = nx.from_pandas_edgelist(domestic_winter_df,'origin_city','dest_city', edge_attr= 'passengers count',
                                       create_using = nx.MultiGraph())

In [ ]:
len(winter_graph.nodes())

1307

Partition

Finding and storing the best partition as json file

In [ ]:
'''winter_partition = community_louvain.best_partition(winter_graph)
community_louvain.modularity(winter_partition,winter_graph)
print(len(set(winter_partition.values())))

#code to save partition as json file
import json
partition_dict = json.dumps(winter_partition)
f = open("intermittent-result/Domestic US/dom_winter_part_7.json", "w")
f.write(partition_dict)
f.close()'''

Retrieving the stored partition

In [ ]:
#code to retrieve partition from json
f = open(os.path.join("intermittent-result/Domestic US/dom_winter_part_7.json"),"r")
dom_winter_partition = json.load(f)
print("Partition length: {}".format(len(dom_winter_partition)))

Partition length: 1307


In [ ]:
winter_mod_value = community_louvain.modularity(dom_winter_partition, winter_graph)
print('Modularity score for winter partition is:{}'.format(winter_mod_value))

Modularity score for winter partition is:0.35323009379206327


Partition to Dataframe

In [ ]:
#partition dictionary to dataframe
dom_winter_partition_df = pd.DataFrame.from_dict(dom_winter_partition, orient='index')
dom_winter_partition_df.reset_index(inplace=True)
dom_winter_partition_df.rename(columns={"index":"city",0:"community"}, inplace=True)
print(dom_winter_partition_df['community'].unique())
dom_winter_partition_df.head()

[0 4 7 6 3 1 2 5]


,city,community
0,aberdeen,0
1,minneapolis,0
2,sioux falls,0
3,abilene,4
4,austin,7


community df creation

In [ ]:
#merge for origin city to community
dom_winter_community_df = pd.merge(dom_winter_partition_df, domestic_winter_df, how='right', left_on='city', right_on='origin_city')


In [ ]:
dom_winter_community_df.isna().sum()

city                0
community           0
Unnamed: 0          0
year                0
month               0
origin_city         0
dest_city           0
month_num           0
passengers count    0
dtype: int64

In [ ]:
dom_winter_community_df.drop(columns=['origin_city','dest_city'], inplace=True)

dom_winter_community_df = dom_winter_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_winter_community_df.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,december,2344.0
1,aberdeen,0,2005,february,2146.0
2,aberdeen,0,2005,january,2391.0
3,aberdeen,0,2006,december,2543.0
4,aberdeen,0,2006,february,2111.0


In [ ]:
#merge for destination city to community
dom_winter_community_df1 = pd.merge(dom_winter_partition_df, domestic_winter_df, how='right', left_on='city', right_on='dest_city')


In [ ]:
dom_winter_community_df1.isna().sum()

city                0
community           0
Unnamed: 0          0
year                0
month               0
origin_city         0
dest_city           0
month_num           0
passengers count    0
dtype: int64

In [ ]:
dom_winter_community_df1.drop(columns=['origin_city','dest_city'], inplace=True)

dom_winter_community_df1 = dom_winter_community_df1.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_winter_community_df1.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,december,2179.0
1,aberdeen,0,2005,february,2167.0
2,aberdeen,0,2005,january,2064.0
3,aberdeen,0,2006,december,2555.0
4,aberdeen,0,2006,february,2003.0


In [ ]:
#concatenating both the above merges
dom_winter_community_df = pd.concat([dom_winter_community_df, dom_winter_community_df1], axis=0)
dom_winter_community_df.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,december,2344.0
1,aberdeen,0,2005,february,2146.0
2,aberdeen,0,2005,january,2391.0
3,aberdeen,0,2006,december,2543.0
4,aberdeen,0,2006,february,2111.0


In [ ]:
dom_winter_community_df = dom_winter_community_df.groupby(['city','community','year','month'], as_index=False)[['passengers count']].sum()
dom_winter_community_df.head()

,city,community,year,month,passengers count
0,aberdeen,0,2005,december,4523.0
1,aberdeen,0,2005,february,4313.0
2,aberdeen,0,2005,january,4455.0
3,aberdeen,0,2006,december,5098.0
4,aberdeen,0,2006,february,4114.0


creating summary df

In [ ]:
dom_agg_dict = {'city':'nunique', 'passengers count':'sum'}
dom_winter_summary_df = dom_winter_community_df.groupby(['community'], as_index=False).agg(dom_agg_dict)
dom_winter_summary_df = dom_winter_summary_df.sort_values(by=['city','passengers count'], ascending=False)
dom_winter_summary_df

,community,city,passengers count
4,4,548,2.784454e+09
7,7,368,1.665126e+09
6,6,299,2.281335e+07
3,3,50,2.859443e+06
0,0,35,1.419879e+08
2,2,3,0.000000e+00
1,1,2,9.046100e+04
5,5,2,4.000000e+00


In [ ]:
dom_winter_summary_df[(dom_winter_summary_df['city']>5) & (dom_winter_summary_df['passengers count']>100)]


,community,city,passengers count
4,4,548,2.784454e+09
7,7,368,1.665126e+09
6,6,299,2.281335e+07
3,3,50,2.859443e+06
0,0,35,1.419879e+08
